**This notebook is copy of this one https://www.kaggle.com/steamedsheep/yolov5-is-all-you-need 
I just changed the following parameters:size10000_conf0.015_augTrue_row.confidence0.28

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm
import sys

sys.path.append('../input/tensorflow-great-barrier-reef')

In [2]:
!cp -r /kaggle/input/bytetrack /kaggle/working/tmp/
!pip install /kaggle/working/tmp/cython_bbox-0.1.3/cython_bbox-0.1.3

!pip install /kaggle/working/tmp/lap-0.4.0/lap-0.4.0
!pip install /kaggle/working/tmp/loguru-0.5.3-py3-none-any.whl
!pip install /kaggle/working/tmp/ninja-1.10.2.2-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl
!pip install /kaggle/working/tmp/thop-0.0.31.post2005241907-py3-none-any.whl
!pip install /kaggle/working/tmp/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar

!pip install /kaggle/working/tmp/onnx-1.8.0-cp37-cp37m-manylinux2010_x86_64.whl
!pip install /kaggle/working/tmp/onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl
!pip install /kaggle/working/tmp/onnx-simplifier-0.3.5/onnx-simplifier-0.3.5

# !pip install /kaggle/working/tmp/flake8_import_order-0.18.1-py2.py3-none-any.whl
# !pip install /kaggle/working/tmp/xmltodict-0.12.0-py2.py3-none-any.whl
# !pip install /kaggle/working/tmp/importlib_metadata-4.2.0-py3-none-any.whl
# !pip install /kaggle/working/tmp/py-cpuinfo-8.0.0/dist/py-cpuinfo-8.0.0.tar
# !pip install /kaggle/working/tmp/pytest_benchmark-3.4.1-py2.py3-none-any.whl
# !pip install /kaggle/working/tmp/motmetrics-1.2.0-py3-none-any.whl

!pip install /kaggle/working/tmp/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar


%cd /kaggle/working
!cp -r ../input/bytetrack/ByteTrack /kaggle/working/
%cd /kaggle/working/ByteTrack
!pip install -e . --no-deps
%cd /kaggle/working/

Processing ./tmp/cython_bbox-0.1.3/cython_bbox-0.1.3
  Preparing metadata (setup.py) ... - \ done
  Created wheel for cython-bbox: filename=cython_bbox-0.1.3-cp37-cp37m-linux_x86_64.whl size=65823 sha256=2a5598d09c58d555f8d90691a1a8c0b604c46700076a95d259c2482634e94109
  Stored in directory: /root/.cache/pip/wheels/ec/69/93/ea052d39657fd8911ad47142ed6afc3a428f25fcd5bbb32a37
Successfully built cython-bbox
Processing ./tmp/lap-0.4.0/lap-0.4.0
  Preparing metadata (setup.py) ... - done
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1665858 sha256=b0019f45b4c663a4d838d9c04e13a9b0437ef5b4082fe0caafc2693d9d668c47
  Stored in directory: /root/.cache/pip/wheels/ab/56/26/a595b29ccd56562a0164b761247c235eccd88e9b838e99d481
Successfully built lap
Processing ./tmp/loguru-0.5.3-py3-none-any.whl
Processing ./tmp/ninja-1.10.2.2-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl
Processing ./tmp/thop-0.0.31.post2005241907-py3-none-any.whl
Processing ./tmp/pycocot

In [3]:
sys.path.append('../input/bytetrack/ByteTrack')
from ByteTrack.yolox.tracker.byte_tracker import BYTETracker

In [4]:
#######################################################
#                      Tracking                       #
#######################################################

# Tracker will update tracks based on detections from current frame
# Matching based on euclidean distance between bbox centers of detections 
# from current frame and tracked_objects based on previous frames

class args:
    det_thresh = 0.20
    track_thresh = 0.10
    track_buffer = 30
    mot20 = False
    match_thresh = 0.3
#     aspect_ratio_thresh = 1.6
    min_box_area = 1000
    
tracker = BYTETracker(args)

# Save frame_id into detection to know which tracks have no detections on current frame
frame_id = 0
#######################################################

In [5]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [6]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [7]:
model = torch.hub.load('../input/yolov5-lib-ds', 
                       'custom', 
                       path='../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt',
                       source='local',
                       force_reload=True)  # local repo
model.conf = 0.20
IMG_SIZE = 3600

YOLOv5 🚀 2022-1-18 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients, 16.2 GFLOPs
Adding AutoShape... 


In [8]:
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    anno = ''
    height, width = img.shape[0], img.shape[1]
    r = model(img, size=IMG_SIZE, augment=False)
    detects = []
    if r.pandas().xyxy[0].shape[0] == 0:
        anno = ''
    else:
        for idx, row in r.pandas().xyxy[0].iterrows():
            if row.confidence > 0.28:
                anno += '{} {} {} {} {} '.format(row.confidence, int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin))
#                 pred.append([row.confidence, row.xmin, row.ymin, row.xmax-row.xmin, row.ymax-row.ymin])
                detects.append([int(row.xmin), int(row.ymin), int(row.xmax), int(row.ymax), row.confidence])

    #######################################################
    #                      Tracking                       #
    #######################################################
    
    # Update tracks using detects from current frame
    if len(detects):
        tracked_objects = tracker.update(np.array(detects), np.array([height, width, frame_id]), np.array([IMG_SIZE, IMG_SIZE]))
        for tobj in tracked_objects:
            # Add objects that have no detections on current frame to predictions
            tlwh = tobj.tlwh
            if tlwh[2] * tlwh[3] > args.min_box_area:
                x_min = int(tlwh[0])
                y_min = int(tlwh[1])
                bbox_width = int(tlwh[2])
                bbox_height = int(tlwh[3])
#                 preds.append([x_min, y_min, bbox_width, bbox_height])
                score = tobj.score
                anno += '{} {} {} {} {} '.format(score, x_min, y_min, bbox_width, bbox_height)
    #         preds.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        #######################################################    
            
    pred_df['annotations'] = anno.strip(' ')
    env.predict(pred_df)
    print('Prediction:', anno.strip(' '))
#     frame_id += 1

0it [00:00, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


2it [00:00,  3.88it/s]

Prediction: 
Prediction: 


3it [00:00,  4.29it/s]

Prediction: 
